# Supervised Learning Example

This notebook demonstrates how to use the SupervisedLearning algorithm for classification and regression tasks.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

from simple_rl.algorithms.supervised_learning import SupervisedLearning

## 1. Classification Example

In [ ]:
# Generate synthetic classification data
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_classes=3,
    random_state=42
)

# Split into train/val/test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)
X_val = torch.FloatTensor(X_val)
y_val = torch.LongTensor(y_val)
X_test = torch.FloatTensor(X_test)
y_test = torch.LongTensor(y_test)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
# Define a simple MLP model for classification
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Create model
model = MLPClassifier(input_dim=20, hidden_dim=64, num_classes=3)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# Create supervised learning algorithm
config = {
    "training": {
        "learning_rate": 0.001,
        "batch_size": 32,
        "num_epochs": 50
    }
}

sl_classifier = SupervisedLearning(
    model=model,
    config=config,
    task_type="classification",
    use_wandb=False
)

# Set data
sl_classifier.create_data_from_arrays(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test
)

In [ ]:
# Train the model
print("Training classifier...")
final_metrics = sl_classifier.train(num_episodes=30)
print(f"\nFinal training metrics: {final_metrics}")

In [ ]:
# Evaluate on test set
test_metrics = sl_classifier.evaluate()
print(f"Test metrics: {test_metrics}")

## 2. Regression Example

In [ ]:
# Generate synthetic regression data
X_reg, y_reg = make_regression(
    n_samples=1000,
    n_features=10,
    n_informative=8,
    noise=0.1,
    random_state=42
)

# Normalize targets
y_reg = (y_reg - y_reg.mean()) / y_reg.std()

# Split into train/val/test
X_train_reg, X_temp_reg, y_train_reg, y_temp_reg = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)
X_val_reg, X_test_reg, y_val_reg, y_test_reg = train_test_split(X_temp_reg, y_temp_reg, test_size=0.5, random_state=42)

# Convert to tensors
X_train_reg = torch.FloatTensor(X_train_reg)
y_train_reg = torch.FloatTensor(y_train_reg).unsqueeze(1)  # Add dimension for MSE loss
X_val_reg = torch.FloatTensor(X_val_reg)
y_val_reg = torch.FloatTensor(y_val_reg).unsqueeze(1)
X_test_reg = torch.FloatTensor(X_test_reg)
y_test_reg = torch.FloatTensor(y_test_reg).unsqueeze(1)

print(f"Train: {X_train_reg.shape}, Val: {X_val_reg.shape}, Test: {X_test_reg.shape}")

In [ ]:
# Define a simple MLP model for regression
class MLPRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, 1)  # Single output for regression
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create model
regression_model = MLPRegressor(input_dim=10, hidden_dim=32)
print(f"Model parameters: {sum(p.numel() for p in regression_model.parameters())}")

In [ ]:
# Create supervised learning algorithm for regression
config_reg = {
    "training": {
        "learning_rate": 0.01,
        "batch_size": 32,
        "num_epochs": 100
    }
}

sl_regressor = SupervisedLearning(
    model=regression_model,
    config=config_reg,
    task_type="regression",
    use_wandb=False
)

# Set data
sl_regressor.create_data_from_arrays(
    X_train_reg, y_train_reg,
    X_val_reg, y_val_reg,
    X_test_reg, y_test_reg
)

In [ ]:
# Train the regression model
print("Training regressor...")
final_metrics_reg = sl_regressor.train(num_episodes=50)
print(f"\nFinal training metrics: {final_metrics_reg}")

In [ ]:
# Evaluate on test set
test_metrics_reg = sl_regressor.evaluate()
print(f"Test metrics: {test_metrics_reg}")

In [ ]:
# Visualize some predictions
sl_regressor.model.eval()
with torch.no_grad():
    predictions = sl_regressor.model(X_test_reg[:100])
    
plt.figure(figsize=(10, 5))
plt.scatter(y_test_reg[:100], predictions, alpha=0.5)
plt.plot([y_test_reg[:100].min(), y_test_reg[:100].max()], 
         [y_test_reg[:100].min(), y_test_reg[:100].max()], 'r--', lw=2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('Regression: True vs Predicted')
plt.show()

## 3. Using with Real Datasets

You can also use SupervisedLearning with real datasets from PyTorch or HuggingFace:

In [ ]:
# Example with PyTorch DataLoader
from torch.utils.data import DataLoader, TensorDataset

# Create your own DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Create new model and algorithm
model2 = MLPClassifier(input_dim=20, hidden_dim=64, num_classes=3)
sl2 = SupervisedLearning(model=model2, config=config, task_type="classification")

# Set data using DataLoaders directly
sl2.set_data(train_data=train_loader, val_data=val_loader)

# Train
# final_metrics2 = sl2.train(num_episodes=10)
# print(f"Final metrics: {final_metrics2}")